# M5: Monitoring, Logs & Final Submission

**Objective:** Monitor the deployed model and submit a consolidated package of all artifacts.

**Tasks:**
1. Basic Monitoring & Logging
2. Model Performance Tracking
3. Final Submission Checklist

---

## 1. Setup

In [1]:
import sys
import os
import requests
import time
import json

sys.path.append(os.path.abspath('..'))
print("✓ Setup complete!")

✓ Setup complete!


## 2. Application Logging

In [15]:
print("Application Logging:")
print("=" * 60)
print("\nLogging Configuration:")
print("  - Format: Structured JSON")
print("  - Level: INFO (configurable)")
print("  - Output: stdout (captured by container runtime)")
print("\nLogged Events:")
print("  - API startup/shutdown")
print("  - Model loading")
print("  - Prediction requests")
print("  - Request latency")
print("  - Errors and exceptions")
print("  - Health check calls")
print("\nExample Log Entry:")
log_entry = {
    "timestamp": "2024-02-10T10:30:45",
    "level": "INFO",
    "message": "Prediction: cat (confidence: 0.92)",
    "latency_seconds": 0.045
}
print(json.dumps(log_entry, indent=2))

Application Logging:

Logging Configuration:
  - Format: Structured JSON
  - Level: INFO (configurable)
  - Output: stdout (captured by container runtime)

Logged Events:
  - API startup/shutdown
  - Model loading
  - Prediction requests
  - Request latency
  - Errors and exceptions
  - Health check calls

Example Log Entry:
{
  "timestamp": "2024-02-10T10:30:45",
  "level": "INFO",
  "message": "Prediction: cat (confidence: 0.92)",
  "latency_seconds": 0.045
}


In [4]:
print("View Logs:")
print("=" * 60)
print("\nLocal (uvicorn):")
print("  # Logs appear in terminal")
print("\nDocker:")
print("=" * 60)
!docker logs cats-dogs-api | head -n 30

print("\nKubernetes:")
print("=" * 60)
!kubectl logs $(kubectl get pods -l app=cats-dogs-classifier -o jsonpath='{.items[0].metadata.name}') | head -n 30
!kubectl logs  deployment/cats-dogs-classifier | head -n 30

View Logs:

Local (uvicorn):
  # Logs appear in terminal

Docker:
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:src.inference_api:Using device: cpu
INFO:src.inference_api:Model loaded from models/model.pt
INFO:src.inference_api:Model loaded successfully
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:42060 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:54920 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:55936 - "GET /health HTTP/1.1" 200 OK
INFO:     192.168.65.1:42280 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:59118 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:35428 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:53940 - "GET /health HTTP/1.1" 200 OK
INFO:     192.168.65.1:23772 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:46460 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:40128 - "GET /health HTTP/1.1" 200 OK
INFO:    

## 3. Prometheus Metrics

In [5]:
# Display Prometheus configuration
with open('../monitoring/prometheus.yml', 'r') as f:
    prom_config = f.read()

print("Prometheus Configuration:")
print("=" * 60)
print(prom_config)

Prometheus Configuration:
global:
  scrape_interval: 15s
  evaluation_interval: 15s

scrape_configs:
  - job_name: 'cats-dogs-classifier'
    static_configs:
      - targets: ['classifier:8000']
    metrics_path: '/metrics'
    scrape_interval: 10s



In [6]:
# Fetch metrics from API (if running)
API_URL = "http://localhost:8000"

try:
    response = requests.get(f"{API_URL}/metrics", timeout=5)
    if response.status_code == 200:
        print("Current Metrics:")
        print("=" * 60)
        print(response.text[:1000])
        print("\n... (truncated) ...")
    else:
        print("Could not fetch metrics")
except:
    print("API not running. Start with: uvicorn src.inference_api:app")

Current Metrics:
# HELP python_gc_objects_collected_total Objects collected during gc
# TYPE python_gc_objects_collected_total counter
python_gc_objects_collected_total{generation="0"} 8311.0
python_gc_objects_collected_total{generation="1"} 790.0
python_gc_objects_collected_total{generation="2"} 60.0
# HELP python_gc_objects_uncollectable_total Uncollectable objects found during GC
# TYPE python_gc_objects_uncollectable_total counter
python_gc_objects_uncollectable_total{generation="0"} 0.0
python_gc_objects_uncollectable_total{generation="1"} 0.0
python_gc_objects_uncollectable_total{generation="2"} 0.0
# HELP python_gc_collections_total Number of times this generation was collected
# TYPE python_gc_collections_total counter
python_gc_collections_total{generation="0"} 698.0
python_gc_collections_total{generation="1"} 63.0
python_gc_collections_total{generation="2"} 5.0
# HELP python_info Python platform information
# TYPE python_info gauge
python_info{implementation="CPython",major="